# Standard Features
## Working with single measurement scans

In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
%matplotlib qt

Qt was found consider switching to qt mode with %matplotlib qt (more comfortable)
Plot_func version 7.3.6
was imported from path:
 C:\Users\jensu\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 c:\Users\jensu\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks


This notebook is an introduction to transient absorption spectroscopy. <br>
In contrast to the other tutorials we are using artificial data for this analysis. <br> 
For this training we will be using 'con_1.SIA' which is clean data with some added noise.<br>
The following dataset 'con_1.SIA' was generated by creating a reaction of the type A->B->C and spectra as shown in this image:
![Chirp](img/Intro_tutorial.png "Data_content")

In [2]:
filenames =['con_1.SIA','con_2.SIA','con_3.SIA','con_4.SIA']               # set name of the file to fit
#If the path is only one folder one can just give the function the name, otherwise one has to give the absolute path
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')  # set path to file to fit
ta=pf.TA(filenames[0],path=filepath)
plt.close('all')
ta.Plot_RAW(0)

In [4]:
ta.Plot_RAW(3)

In [5]:
# Can't remember the commands?
ta()


		Hello, this is a transient absorption project, 
		this project contains all the parameter and
		the main functions you can use to analyze data.
		-----------------------------------------------
		The main functions are:
		Plotting: ta.Plot_RAW(), ta.Plot_Interactive(), ta.Plot_fit_output()
		Comparing: ta.Compare_at_time(), ta.Compare_at_wave(), ta.Compare_DAC()
		Saving: ta.Save_Powerpoint(), ta.Save_project(), ta.Save_data(), ta.Save_Plots()
		Fitting: ta.Fit_Global()
		Shaping: ta.Background(), ta.Cor_Chirp(), ta.Man_Chirp()
		Parameter: call "ta.__dict__.keys()" to see all implemented shaping parameter
		-------------------------------------------------------------------------------
		you can get help and inspiration how to use each function by typing the function 
		name followed by a question mark like "ta.Fit_Global?"
		or by going to the documentation webpage:
		https://kimopack.readthedocs.io/en/latest
		----------------------------------------------------------------------

We investigate the 2D matrix using the option to click on the data. We plot this spectrum twice, once in logaritmic scale and once in linear scale. Making the plot clickable means that the values where you click are shown. 
I usually use this method to define where I do would like to show the plots.

In [3]:
#the 2d Matrix is plot 0 (check ta.Plot_RAW? for a tutorial)
ta.log_scale=False # This is the default
ta.Plot_RAW(0,print_click_position=True)
ta.log_scale=True   # This scales the 2d plot into log scale.
ta.Plot_RAW(0,print_click_position=True)

Setting these values I plot the same matrix again and select the interesting wavelength using the same process

In [7]:
ta.bordercut=[400,975]
ta.timelimits=[-0.2,500]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

Based on the shown wavelength I set the interesting wavelength and plot it a last time and select the intersting time points

In [8]:
ta.rel_wave=[430,487,525,640,720,820,900,950]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

again, the clicked points are converted to time points and we have set the intersting time points.

In [9]:
ta.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]

Now we are ready to make an overview plot of the data

In [10]:
ta.log_scale=False
ta.Save_Powerpoint(save_Fit=False,title='Tutorial plot')

The images and a powerpoint was saved to c:\Users\jensu\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks\Data\Introduction\result_figures


For fitting usually the kinetic plots are a good choice to select the times I believe are a good starting point for a fit. In this process i look for all the clear decays and click on them. I also look for if there is signal before time=0 (nothing here) and if there is a signal after all is decayed (nothing here)

In [11]:
ta.Plot_RAW(1,print_click_position=True)

Based on these clicks we create a model. In general starting with exponential decays is a good idea. Here we choose three distinct times in lines 4-6, add instrument parameter (fixed for now) in rows 8 and 9. 
Important is the line 10-12. This is a simple trick to "freeze" all parameter. This allows us to check how good are our starting parameter.
Line 14 is triggering the fit. And line 17 plotting the results

In [4]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 1:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

ATTENTION: we have not optimized anything but just returned the parameters

Fit Results:
Model Used: exponential

The minimum error is:9.95891241e-03
The minimum R2-value is:9.71664083e-01

In Rates

+------------+----------+--------------+--------+-------+-------+--------+
|            |    value |   init_value | vary   |   min |   max | expr   |
+============+==========+==============+========+=======+=======+========+
| k0         | 7.14286  |     7.14286  | False  |  0    | inf   |        |
+------------+----------+--------------+--------+-------+-------+--------+
| k1         | 0.425532 |     0.425532 | False  |  0    | inf   |        |
+------------+----------+--------------+--------+-------+-------+--------+
| k2         | 0.025    |     0.025    | False  |  0    | inf   |        |
+------------+----------+--------------+--------+-------+-------+--------+
| t0         | 0        |     0        | False  | -2    |   2   |        |
+------------+----------+--------------+--------+-

In [14]:
species=pf.Species_Spectra(ta)

In [15]:
for key in species.keys():
    ta.Plot_RAW(0,ds=species[key])

As everything looks pretty good we simply copy paste the same code, but now set the if switch in line 9 to "0" to disable this loop and premit the optimization 

In [11]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output


Fit Results:
Model Used: exponential

The minimum error is:1.24774594e-03
The minimum R2-value is:9.96386956e-01

In Rates

+------------+------------+--------------+--------+-------+-------+--------+
|            |      value |   init_value | vary   |   min |   max | expr   |
+============+============+==============+========+=======+=======+========+
| k0         | 40.244     |     7.14286  | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k1         |  0.40628   |     0.425532 | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k2         |  0.0247817 |     0.025    | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| t0         |  0         |     0        | False  | -2    |   2   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| resolution |  0.086     | 

The result is refining and getting much better, but the change to the starting values is very small. So we repeat the same fit but allow the time resolution to be adjusted optimizing the instrument parameter. (line 6 and 7)

In [12]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=True)       # Allow the instrument response to adjust (False here)

ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

we use adaptive mode for nelder

Fit Results:
Model Used: exponential

The minimum error is:7.97693666e-04
The minimum R2-value is:9.97690153e-01

In Rates

+------------+------------+--------------+--------+-------+-------+--------+
|            |      value |   init_value | vary   |   min |   max | expr   |
+============+============+==============+========+=======+=======+========+
| k0         | 24.6422    |     7.14286  | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k1         |  0.411769  |     0.425532 | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k2         |  0.0248365 |     0.025    | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| t0         | -0.0183704 |     0        | True   | -2    |   2   |        |
+------------+------------+--------------+--------+-------+-------+------

From this we fint and fix the instrument resolution and the arrival time of the laser.

In [13]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output


Fit Results:
Model Used: exponential

The minimum error is:1.24774594e-03
The minimum R2-value is:9.96386956e-01

In Rates

+------------+------------+--------------+--------+-------+-------+--------+
|            |      value |   init_value | vary   |   min |   max | expr   |
+============+============+==============+========+=======+=======+========+
| k0         | 40.244     |     7.14286  | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k1         |  0.40628   |     0.425532 | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k2         |  0.0247817 |     0.025    | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| t0         |  0         |     0        | False  | -2    |   2   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| resolution |  0.086     | 

A quick look on the DAS it is clear that they are the change of the spectra, to get species associated spectra we change to target analysis and the related model A->B->C by changing the model to 'consecutive' but keeping the same parameter.

In [14]:
plt.close('all')
ta.Plot_fit_output(0) 

In [15]:
ta.mod='consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=-0.01837,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output(0)                            # plot the fit output
ta.Plot_fit_output(4)                           # plot the fit output


Fit Results:
Model Used: consecutive

The minimum error is:9.61546885e-04
The minimum R2-value is:9.97215690e-01

In Rates

+------------+------------+--------------+--------+-------+-------+--------+
|            |      value |   init_value | vary   |   min |   max | expr   |
+============+============+==============+========+=======+=======+========+
| k0         | 40.57      |     7.14286  | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k1         |  0.406814  |     0.425532 | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k2         |  0.0247695 |     0.025    | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| t0         | -0.01837   |    -0.01837  | False  | -2    |   2   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| resolution |  0.086     | 

the longer timepoints are very well represented, but there are still some errors in the early times. So we permit the laser arrival time, and the instrument response function free again and change the model to "full_consecutive". (In case you are in a hurry, set the vary of 't0' and 'resolution' to False. 

In [16]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                     
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.Save_project('full_consecutive_fit.hdf5')
# saved about 45s

ta=pf.TA('full_consecutive_fit.hdf5',path=ta.path)
ta.Print_Results()
plt.close('all')
ta.Plot_fit_output(0)                            # plot the fit output
ta.Plot_fit_output(4)

all chirp coefficients are zero so no chirp correction applied

Fit Results:
Model Used: full_consecutive

The minimum error is:8.55942977e-04
The minimum R2-value is:9.97521483e-01

In Rates

+------------+------------+--------------+--------+-------+-------+--------+
|            |      value |   init_value | vary   |   min |   max | expr   |
+============+============+==============+========+=======+=======+========+
| k0         | 24.1231    |     7.14286  | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k1         |  0.394108  |     0.425532 | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| k2         |  0.0238342 |     0.025    | True   |  0    | inf   |        |
+------------+------------+--------------+--------+-------+-------+--------+
| t0         |  0         |     0        | False  | -2    |   2   |        |
+------------+------------+----------

This results in a 60 percent improvement of the R2 factor. But still all the DAS contain a contribution of the ground state bleach. So we are adding the ground state explicitely to try to separate the contributions. We lock the resolution and the laser arrival time "I0" to speed up the fit

In [17]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')

ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.Save_project('full_consecutive_fit_with_GS.hdf5')
# saved about 45s
reload(pf)
ta=pf.TA('full_consecutive_fit_with_GS.hdf5',path=ta.path)
ta.Print_Results()
plt.close('all')
ta.Plot_fit_output(4)                            # plot the fit output
ta.Plot_fit_output(0)

Qt was found consider switching to qt mode with %matplotlib qt (more comfortable)
We need python-pptx to create a powerpoint file. Not essential. Either use pip or for anaconda: conda install -c conda-forge python-pptx
Plot_func version 7.1.21
was imported from path:
 C:\Users\Jens\anaconda3\Lib\site-packages\KiMoPack
The current working folder is:
 d:\Dropbox\coding\github\KiMoPack\Tutorial_Notebooks
all chirp coefficients are zero so no chirp correction applied

Fit Results:
Model Used: full_consecutive

The minimum error is:6.73796113e-04
The minimum R2-value is:9.98048918e-01

In Rates

+-------------+----------+--------------+--------+---------+-------+--------+
|             |    value |   init_value | vary   |     min |   max | expr   |
+=============+==========+==============+========+=========+=======+========+
| k0          | 9.98572  |     9.98572  | True   |    0    | inf   |        |
+-------------+----------+--------------+--------+---------+-------+--------+
| k1        

C:\Users\Jens\anaconda3\Lib\site-packages\ipykernel\eventloops.py:113: UserWarning: All values for SymLogScale are below linthresh, making it effectively linear. You likely should lower the value of linthresh. 
  app.exec_()


In [24]:
#plt.close('all')
ta.Fit_Global(ext_spectra)

C:\Users\Jens\anaconda3\Lib\site-packages\ipykernel\eventloops.py:113: UserWarning: All values for SymLogScale are below linthresh, making it effectively linear. You likely should lower the value of linthresh. 
  app.exec_()


Now we can compare the time evolution of the components with the species associated spectra that were put into the data.
![Chirp](img/Intro_tutorial.png "Data_content")

It is a very imporant step to check the confidence interval, that unfortunately does take quite some time. (typically 100x the time for  a single optimization. It is generally a good idea to save the project before you do that with ta.Save_project() to not loose the prior work. The following cell shows the result of this run. The file "con_1_solved.hdf5" contains the project with the result for you to inspect.

In [ ]:
#ta.Fit_Global(confidence=0.95)
#ta.Save_project('con_1_solved.hdf5')

In [ ]:
#ta.Fit_Global(confidence=0.95)
#ta.Save_project('con_1_solved.hdf5')
#Saved 20min

ta=pf.TA('con_1_solved.hdf5',path=ta.path)
ta.filename='Solved_file.hdf5'
ta.Print_Results()

In [ ]:
# save the results
plt.close('all')
ta.Save_Powerpoint(title='Tutorial plot after Fit')

## Real data

In real data, the measured signals are not as nice and clear as we have worked with up to now.<br>
For 'con_2.SIA','con_3.SIA','con_4.SIA','con_5.SIA' 'con_6.SIA' typical disturbances were introduced. <br>
for which additional complications such as noise, chirp and crossphase modulation was added.<br>

1. Use the function "Cor_chirp" that is part of the ta object to correct the chrip in "con_2.SIA".
1. Apply the same chirp correction (either via the file name or the ta.fitcoeff to the following files. You do want to use ta.intensity range and ta.log_plot=True/False to make the development in this file visible
1. In file con_3.SIA you additionally need to adress the spectral limits using "bordercut"
1. In file con_4.SIA we have typical artifacts and Cross-Phase-Modulation. Use "ignore_time_region" to blind this out 
1. in File con_5.SIA we have to additionally reject a spectral region in which the pump laser light scattered into the detector, as is often the case if measuring e.g. nano particles.
1. In the final file con_6.SIA our initial state has some Frank condon type oscillations. Fit the data with the kinetics, and find the oscillations in the Plot_Fit_output.

In [ ]:
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')

In [ ]:
ta1=pf.TA('con_2.SIA',path=filepath)
ta1.intensity_range=0.005
ta1.log_scale=True
ta1.Plot_RAW(0)

In [ ]:
ta1.Cor_Chirp()

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]


In [ ]:
ta1=pf.TA('con_3.SIA',path=filepath)
ta1.Cor_Chirp(chirp_file='con_2_chirp.dat')

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.Plot_RAW(0)

In [ ]:
ta1.bordercut=[390,1150]

In [ ]:
ta1=pf.TA('con_4.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
plt.close('all')
ta1.Plot_RAW(0)


In [ ]:
ta1.ignore_time_region=[-0.15,0.1]

In [ ]:
ta1=pf.TA('con_5.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.ignore_time_region=[-0.15,0.1]
plt.close('all')
ta1.Plot_RAW(0)

In [ ]:
ta1.scattercut=[525,580]

In [ ]:
reload(pf)

In [ ]:
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')
ta1=pf.TA('con_6.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=False
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.scattercut=[525,580]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.ignore_time_region=[-0.15,0.1]
plt.close('all')
ta1.Plot_RAW(0)

In [ ]:
#ta1.ds=ta1.ds-ta.re['AC']
ta1.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)
par.add('explicit_GS')
ta1.par=par
if 1:
    for key in par.keys():
        par[key].vary=False
ta1.Fit_Global()
#ta1.Save_project()
#ta1=pf.TA(ta1.filename,path=ta1.path)
plt.close('all')
ta1.Plot_fit_output(0)
ta1.Plot_fit_output(4)

In [ ]:
plt.close('all')
ta1.Plot_fit_output(1)
ta1.Plot_fit_output(2)

In [ ]:
ta2=ta1.Copy()
ta2.ds=ta1.re['AE']

In [ ]:
dicten=pf.Species_Spectra(ta1)
keys=list(dicten.keys())
dicten.keys()

In [ ]:
ta2.ds=ta1.re['A']-dicten[1]-dicten[2]-dicten['GS']
ta2.intensity_range=3e-4
ta2.rel_wave=[620,700,740,800,830,860]
ta2.timelimits=[0.1,10]
plt.close('all')
ta2.Plot_RAW(0)
ta2.Plot_RAW(1,scale_type='linear')

## The principle behind the modelling repetition

1. Create vectors that represent the species that develop

    1. Choose a model
    1. Set the parameter of the model
    1. For independent exponential decays convolute instrument response (here a broadened heavyside step function) with one exponential decay using the rate parameter
    1. for any other from me provided function: We create a reponse "pulse", Write a differential equation and stepwise calculate the concentration by evaluating the rate equation
    1. to avoid artifacts: sub-sample
    

In [ ]:
def manual_consecutive(times,pardf):								
	c=np.zeros((len(times),3),dtype='float') 						#creation of matrix that will hold the concentrations
	g=gauss(times,sigma=pardf['resolution']/FWHM,mu=pardf['t0']) 	#creating the gaussian pulse that will "excite" our sample
	sub_steps=10 													#defining how many extra steps will be taken between the main time_points
	for i in range(1,len(times)):									#iterate over all timepoints
		dc=np.zeros((3,1),dtype='float')							#the initial change for each concentration, the "3" is representative of how many changes there will be
		dt=(times[i]-times[i-1])/(sub_steps)						# as we are taking smaller steps the time intervals need to be adapted
		c_temp=c[i-1,:]												#temporary matrix holding the changes (needed as we have sub steps and need to check for zero in the end)
		for j in range(int(sub_steps)):
			dc[0]=-pardf['k0']*dt*c_temp[0]+g[i]*dt					#excite a small fraction with g[i] and decay with 'k0'
			dc[1]=pardf['k0']*dt*c_temp[0]-pardf['k1']*dt*c_temp[1]	#form with "k0" and decay with "k1"
			dc[2]=pardf['k1']*dt*c_temp[1]-pardf['k2']*dt*c_temp[2]	#form with "k1" and decay with "k2"
			for b in range(c.shape[1]):
				c_temp[b] =np.nanmax([(c_temp[b]+float(dc[b])),0.])		#check that nothing will be below 0 (concentrations)
		c[i,:] =c_temp												#store the temporary concentrations into the main matrix
	c=pandas.DataFrame(c,index=times)								#write back the right indexes
	c.index.name='time'												#and give it a name
	c.columns=['1MLCT','3MLCT','3MC']								#this is optional but very useful. The species get names that represent some particular states
	if 'background' in list(pardf.index.values):					#optional but usefull, allow the keyword "background" to be used to fit the background in the global analysis
		c['background']=1											#background always there (flat)
	return c														#return the concentrations to the global fitting

In [ ]:
import function_library as func
reload(func)

In [ ]:
ta1.mod=func.P12
#ta1.Fit_Global()

In [ ]:
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)
ta1.timelimits([1,1000])

In [ ]:
import scipy.constants as const
import numpy as np
dicten={
    'IR-1100':{'water':1/(const.c/1.325),
               'glas':1/(const.c/1.449),
               'BBO':1/(const.c/1.654),
               'CaF':1/(const.c/1.428)},
'UV-350':{'water':1/(const.c/1.343),
          'glas':1/(const.c/1.477),
          'BBO':1/(const.c/1.707),
          'CaF':1/(const.c/1.447)}}

In [ ]:
IR=1e-3*dicten['IR-1100']['water']#+1e-3*dicten['IR-1100']['glas']
UV=1e-3*dicten['UV-350']['water']#+1e-3*dicten['UV-350']['glas']
print('%.0f fs mismatch'%((UV-IR)*1e15))
print('%.0f fs total for 35fs laser'%(np.sqrt((35e-15**2)*2+(60e-15**2))*1e15))
print('%.0f fs total for 90fs laser'%(np.sqrt((75e-15**2)*2+(60e-15**2))*1e15))

In [ ]:
IR=0.2e-3*dicten['IR-1100']['water']#+1e-3*dicten['IR-1100']['glas']
UV=0.2e-3*dicten['UV-350']['water']#+1e-3*dicten['UV-350']['glas']
print('%.0f fs mismatch'%((UV-IR)*1e15))
print('%.0f fs total for 35fs laser'%(np.sqrt((35e-15**2)*2+(12e-15**2))*1e15))